In [ ]:
%pip install torch==2.1.0 transformers peft trl accelerate bitsandbytes datasets

In [ ]:
%pip install huggingface_hub

In [ ]:
from huggingface_hub import login

login(token="")

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
dataset_name = "lora_dataset.jsonl"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj",
    "lm_head",
]

In [ ]:
output_dir = "./results"
num_train_epochs = 3
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
gradient_checkpointing = True
optim = "paged_adamw_8bit"
learning_rate = 3e-4
weight_decay = 0.001
fp16 = True
bf16 = False
max_grad_norm = 0.3
max_steps = -1
warmup_ratio = 0.03
lr_scheduler_type = "cosine"
logging_steps = 25
save_steps = 1000

In [ ]:
max_seq_length = 512
packing = False

In [ ]:
dataset = load_dataset("json", data_files=dataset_name, split="train")

In [ ]:
def filter_short_completions(example):
    completion = example["completion"].strip()
    return len(completion.split()) >= 3

dataset = dataset.filter(filter_short_completions)

In [ ]:
compute_dtype = torch.float16

bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bitsandbytes_config,
    device_map={ "": 0 }
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=lora_target_modules,
)
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    gradient_checkpointing=gradient_checkpointing,
)

In [ ]:
def format_data(example):
    return {"text": example["prompt"] + example["completion"]}

dataset = dataset.map(format_data)

In [ ]:
sft_trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_arguments,
)

In [ ]:
sft_trainer.train()

In [ ]:
sft_trainer.model.save_pretrained(output_dir)